In [1]:
import pandas
import numpy as np
from utils import *
import mne
import matplotlib.pyplot as plt
from os import walk
# from sklearn.decomposition import FastICA
# from sklearn.naive_bayes import GaussianNB, MultinomialNB
# from sklearn.model_selection import cross_val_score
# from itertools import combinations
# from sklearn.preprocessing import normalize
# from sklearn.utils import shuffle
import pickle

from multiprocessing import Pool
plt.style.use('seaborn-whitegrid')

# GroudTruth

In [ ]:
gt = load_groudtruth('./HEXACO.csv')
fig, ax = plt.subplots()
ax.bar(['H','E','X','A','C','O'],[ sum(gt['lh']), sum(gt['le']), sum(gt['lx']), sum(gt['la']), sum(gt['lc']), sum(gt['lo'])  ])
ax.set_title("Distribution of Labels")
ax.set_xlabel("Labels")
# ax.set_ylabel("Distribution of Labels")
plt.show()

In [3]:
EEG_audio, EEG_image = load_data(path='./data',exclude_list=[11,36])

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html